### Imports

In [8]:
import numpy as np
from astropy.table import Table

### Data

In [9]:
mangaHI = Table.read('input/mangaHIall.fits', format='fits')
alfaHI = Table.read('input/manga_dr15_alfalfa.fits', format='fits')
manga = Table.read('input/Pipe3D-master_file_vflag_10_smooth2p27_N2O2_noWords.txt', format='ascii.commented_header')
kias7 = Table.read( 'input/kias1033_5_P-MJD-F_MPAJHU_ZdustOS_stellarMass_BPT_SFR_NSA_vflag.txt', format='ascii.commented_header')

## Add HI data to MaNGA table

##### Initialize new columns in MaNGA data table

In [10]:
manga['WF50'] = np.NaN*np.ones(len(manga), dtype=float)
manga['WP20'] = np.NaN*np.ones(len(manga), dtype=float)
manga['logHI'] = np.NaN*np.ones(len(manga), dtype=float)

##### Add ALFALFA HI data

In [11]:
for i in range(len(alfaHI)):
    
    galID = alfaHI['PLATEIFU'][i]
    plate, IFU = galID.split('-')
    
    # Find the corresponding row index for this galaxy in the MaNGA data table
    plate_boolean = manga['MaNGA_plate'] == int(plate)
    IFU_boolean = manga['MaNGA_IFU'] == int(IFU)
    manga_boolean = np.logical_and(plate_boolean, IFU_boolean)
    
    # Add data to MaNGA data table
    manga['WF50'][manga_boolean] = alfaHI['WF50'][i]
    manga['WP20'][manga_boolean] = alfaHI['WP20'][i]
    manga['logHI'][manga_boolean] = alfaHI['LOGMHI'][i]

##### Add GBT HI data

In [12]:
for i in range(len(mangaHI)):
    
    galID = mangaHI['plateifu'][i]
    plate, IFU = galID.split('-')
    
    # Find the corresponding row index for this galaxy in the MaNGA data table
    plate_boolean = manga['MaNGA_plate'] == int(plate)
    IFU_boolean = manga['MaNGA_IFU'] == int(IFU)
    manga_boolean = np.logical_and(plate_boolean, IFU_boolean)
    
    # Add data to MaNGA data table
    manga['WF50'][manga_boolean] = mangaHI['WF50'][i]
    manga['WP20'][manga_boolean] = mangaHI['WP20'][i]
    manga['logHI'][manga_boolean] = mangaHI['logMHI'][i]

## Isolate those MaNGA galaxies with HI detections

We only want to analyze those galaxies in MaNGA with HI detections, so let's create a subset of the main MaNGA data table that contains only those galaxies with HI detections.

In [13]:
HI_boolean = np.logical_or.reduce((np.isfinite(manga['WF50']), 
                                   np.isfinite(manga['WP20']), 
                                   np.isfinite(manga['logHI'])))

manga_HI = manga[HI_boolean]

In [14]:
manga_HI.write('input/gbt_alfalfa_HIdata.txt', format = 'ascii.commented_header', overwrite = True)